In [ ]:
import numpy as np
import os
from tensorflow.keras.utils import to_categorical

class NpyDataGenerator:
    def __init__(self, npy_dir, batch_size, shuffle=True):
        self.npy_dir = npy_dir
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.class_names = sorted(os.listdir(npy_dir)) # get the list of subdirectories in the npy_dir as class names
        self.num_classes = len(self.class_names) # count the number of classes
        self.class_indices = dict(zip(self.class_names, range(self.num_classes))) # create a dictionary mapping class names to class indices
        self.npy_files = [] # create an empty list for storing the npy files
        for class_name in self.class_names:
            class_dir = os.path.join(npy_dir, class_name) # get the path to the subdirectory for this class
            class_files = os.listdir(class_dir) # get the list of npy files in this subdirectory
            for npy_file in class_files:
                self.npy_files.append((os.path.join(class_dir, npy_file), class_name)) # append a tuple containing the path to the npy file and its corresponding class name
        self.num_samples = len(self.npy_files) # count the number of samples
        self.indices = np.arange(self.num_samples) # create an array of indices for shuffling
        self.on_epoch_end() # shuffle the indices

    def __len__(self):
        return int(np.ceil(self.num_samples / self.batch_size)) # calculate the number of batches per epoch

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size] # get the indices for the current batch
        batch_npy_files = [self.npy_files[k] for k in batch_indices] # get the filenames for the current batch
        batch_x = [] # create an empty list for the input data
        batch_y = [] # create an empty list for the target data
        for npy_file, class_name in batch_npy_files:
            npy_data = np.load(npy_file) # load the npy file
            x = npy_data[..., :-1]  # extract the input data from the npy file
            y = self.class_indices[class_name] # get the class index for the current sample
            batch_x.append(x) # add the input data to the current batch
            batch_y.append(y) # add the class index to the current batch
        batch_y = to_categorical(batch_y, num_classes=self.num_classes) # one-hot encode the class indices
        return np.array(batch_x), np.array(batch_y) # return the batch as a tuple of input and target data

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices) # shuffle the indices at the end of each epoch to randomize the order of samples
